# Moscow Marathon Full Results 2018


## 1_full_results_mm_2018.csv

Columns : {Bib, finish_time_sec, finish_time_result, race, pace_sec, pace(minpkm), pace(kmph), half_pace_sec, half_pace(minpkm), half_pace(kmph), gender_en, agev name_en, location_city_ru, location_city_en, country_code_alpha_3, flag_DNF, flag_all_split_exist, race_uniform_index}

Data : {1, 8911, 2h 28min 31sec, 42.195 km, 211.1861595, 3:31 min/km 17.0 km/h, 208.3185212, 3:28 min/km, 17.3 km/h, Female, 30, Sardana Trofimova, –Ø–∫—É—Ç—Å–∫, Yakutsk, RUS, 0, 1, 0.000132899}

## 1_split_results_mm_2018.csv

Columns : {bib, split_name, split, split_time_sec, split_time_result, split_pace_sec, split_pace(minpkm), split_pace(kmph), split_uniform_index}

Data : {11, Kirui, Geoffrey, 24, M, Keringet, KEN, 0:15:25, 0:30:28, 0:45:44, 1:01:15, 1:04:35, 1:16:59, 1:33:01, 1:48:19, 2:02:53, 0:04:57, - 2:09:37, 1, 1, 11}

In [1]:
from datetime import datetime
import numpy as np
import pandas as pd

# 데이터 불러오기
full_df = pd.read_csv('./data/1_full_results_mm_2018.csv')
split_df = pd.read_csv('./data/1_split_results_mm_2018.csv')

# bib 통일
full_df['bib'] = full_df['bib'].astype(str)
split_df['bib'] = split_df['bib'].astype(str)

# --------------------------
# split_time_sec pivot (5K~40K만)
split_time = split_df.pivot_table(index='bib', columns='split_name', values='split_time_sec')

# Half marathon, Marathon 열 제거
split_time = split_time.drop(columns=['Half marathon', 'Marathon'], errors='ignore')

# ' km' 제거 후 'K' 붙이기
split_time.columns = [col.replace(' km', '') + 'K' for col in split_time.columns]
split_time = split_time.apply(pd.to_numeric, errors='coerce')
split_time.reset_index(inplace=True)

# --------------------------
# split_pace_sec pivot (5p~40p만)
split_pace = split_df.pivot_table(index='bib', columns='split_name', values='split_pace_sec')
split_pace = split_pace.drop(columns=['Half marathon', 'Marathon'], errors='ignore')

split_pace.columns = [col.replace(' km', '') + 'p' for col in split_pace.columns]
split_pace = split_pace.apply(pd.to_numeric, errors='coerce')
split_pace.reset_index(inplace=True)

# --------------------------
# 필요한 컬럼 선택 및 전처리
reduced_df = full_df[['bib', 'age', 'gender_en', 'country_code_alpha_3', 'finish_time_sec']].copy()
reduced_df.rename(columns={
    'bib': 'Bib',
    'age': 'Age',
    'gender_en': 'M/F',
    'country_code_alpha_3': 'Country',
    'finish_time_sec': 'Final_Time'
}, inplace=True)

# Final_Time이 NaN인 경우 제거
reduced_df = reduced_df.dropna(subset=['Final_Time'])

# Age_group (19이하 → 19, ..., 70 이상 → 70)
def age_group(age):
    if age <= 19:
        return 19
    elif age >= 70:
        return 70
    else:
        return (age // 5) * 5 + 4  # 20~24 → 24, 25~29 → 29, ...

reduced_df['Age_group'] = reduced_df['Age'].apply(age_group)

# M/F: Male → 0, Female → 1
reduced_df['M/F'] = reduced_df['M/F'].map({'Male': 0, 'Female': 1})

# Sub 그룹핑
def set_sub_group(seconds):
    if pd.isna(seconds):
        return pd.NA
    hours = seconds / 3600
    if hours < 3:
        return 3
    elif hours < 4:
        return 4
    elif hours < 5:
        return 5
    elif hours < 6:
        return 6
    else:
        return 7

reduced_df['Sub'] = reduced_df['Final_Time'].apply(set_sub_group).astype('Int64')

# --------------------------
# 병합
merged = reduced_df.merge(split_time, how='left', left_on='Bib', right_on='bib')
merged = merged.merge(split_pace, how='left', on='bib')
merged.drop(columns=['bib'], inplace=True)

# Dataset 컬럼 추가
merged['Dataset'] = 'M'

# --------------------------
# 컬럼 순서 지정 (5~40K, 5~40p만 포함)
pace_cols = [f'{k}p' for k in range(5, 45, 5)]
time_cols = [f'{k}K' for k in range(5, 45, 5)]

columns_order = ['Bib', 'Age_group', 'M/F', 'Country'] + \
                pace_cols + ['Final_Time', 'Sub'] + \
                time_cols + ['Dataset']

# 존재하는 컬럼만 유지
columns_order = [col for col in columns_order if col in merged.columns]

# 결측치 제거
Moscow_df = merged[columns_order].dropna()

# Bib 재설정
Moscow_df['Bib'] = range(1, len(Moscow_df) + 1)
cols = Moscow_df.columns.tolist()
cols.remove('Bib')
Moscow_df = Moscow_df[['Bib'] + cols]

# int 변환
int_cols = Moscow_df.columns.difference(['Country', 'Dataset'])
Moscow_df[int_cols] = Moscow_df[int_cols].astype(int)

# 저장
Moscow_df.to_csv('./data/Moscow_Marathon_Processed.csv', index=False)

# 확인
print(Moscow_df.head())

FileNotFoundError: [Errno 2] No such file or directory: './data/1_full_results_mm_2018.csv'

# Finishers Boston Marathon 2015, 2016 & 2017

In [ ]:
import pandas as pd
import numpy as np

# CSV 파일 로드
df_15 = pd.read_csv('./data/marathon_results_2015.csv')
df_16 = pd.read_csv('./data/marathon_results_2016.csv')
df_17 = pd.read_csv('./data/marathon_results_2017.csv')

# 연도 컬럼 추가
df_15['Year'] = 2015
df_16['Year'] = 2016
df_17['Year'] = 2017

# 데이터 통합
df = pd.concat([df_15, df_16, df_17], ignore_index=True)

# 불필요한 컬럼 제거
drop_cols = ['Unnamed: 0', 'Unnamed: 8', 'Unnamed: 9', 'State', 'Citizen', 'Proj Time']
df = df.drop(columns=[col for col in drop_cols if col in df.columns])

# 시간형 컬럼 처리 대상: 21K는 제거 대상
time_cols = ['5K', '10K', '15K', '20K', '25K', '30K', '35K', '40K', 'Pace', 'Official Time']
for col in time_cols:
    df[col] = pd.to_timedelta(df[col], errors='coerce')

# 초 단위로 변환
for col in ['5K', '10K', '15K', '20K', '25K', '30K', '35K', '40K', 'Official Time']:
    df[col] = df[col].dt.total_seconds()

# 컬럼명 변경
df.rename(columns={'Official Time': 'Final_Time'}, inplace=True)

# 페이스 계산 (21K 제외)
distance_km = {'5K': 5, '10K': 10, '15K': 15, '20K': 20, '25K': 25, '30K': 30, '35K': 35, '40K': 40}
for dist, km in distance_km.items():
    pace_col = dist.replace('K', 'p')
    df[pace_col] = df[dist] / km

# Age_group 지정
def age_group(age):
    if age < 20:
        return 19
    elif age < 25:
        return 24
    elif age < 30:
        return 29
    elif age < 35:
        return 34
    elif age < 40:
        return 39
    elif age < 45:
        return 44
    elif age < 50:
        return 49
    elif age < 55:
        return 54
    elif age < 60:
        return 59
    elif age < 65:
        return 64
    elif age < 70:
        return 69
    else:
        return 70

df['Age_group'] = df['Age'].apply(age_group)

# 성별 인코딩
df['M/F'] = df['M/F'].map({'M': 0, 'F': 1})

# Sub (시간 그룹)
def sub_group(time_sec):
    hours = time_sec / 3600
    if hours <= 2:
        return 3
    elif hours <= 3:
        return 4
    elif hours <= 4:
        return 5
    elif hours <= 5:
        return 6
    else:
        return 7

df['Sub'] = df['Final_Time'].apply(sub_group)

# 필요한 컬럼만 추출
base_cols = ['Bib', 'Age_group', 'M/F', 'Country']
pace_cols = [k.replace('K', 'p') for k in distance_km.keys()]
time_cols = list(distance_km.keys())
final_cols = ['Final_Time', 'Sub']
df['Dataset'] = 'B'

ordered_cols = base_cols + pace_cols + final_cols + time_cols + ['Dataset']
df = df[ordered_cols]

# 결측치 제거 및 Bib 재할당
df = df.dropna()
df.reset_index(drop=True, inplace=True)
df['Bib'] = df.index + 1

# 숫자형 컬럼 int로 변환
int_cols = df.columns.difference(['Country', 'Dataset'])
df[int_cols] = df[int_cols].astype(int)

# 저장
df.to_csv('./data/boston_processed.csv', index=False)

# 결과 확인
print(df.head())

   Bib  Age_group  M/F Country   5p  10p  15p  20p  25p  30p  ...  Sub   5K  \
0    1         29    0     ETH  176  178  179  181  182  184  ...    4  883   
1    2         34    0     ETH  176  178  179  181  182  183  ...    4  883   
2    3         29    0     KEN  176  178  179  181  182  184  ...    4  883   
3    4         29    0     KEN  176  178  180  181  182  184  ...    4  883   
4    5         34    0     KEN  176  178  179  181  182  184  ...    4  883   

    10K   15K   20K   25K   30K   35K   40K  Dataset  
0  1783  2697  3629  4567  5520  6479  7359        B  
1  1783  2698  3628  4567  5519  6479  7362        B  
2  1783  2697  3629  4567  5520  6479  7381        B  
3  1784  2701  3629  4567  5520  6483  7427        B  
4  1784  2698  3628  4567  5520  6479  7407        B  

[5 rows x 23 columns]


In [ ]:
import pandas as pd

# 파일 경로
boston_path = './data/boston_processed.csv'
moscow_path = './data/Moscow_Marathon_Processed.csv'

# 데이터 불러오기
df_boston = pd.read_csv(boston_path)
df_moscow = pd.read_csv(moscow_path)

# 병합 (인덱스 초기화)
df_merged = pd.concat([df_boston, df_moscow], ignore_index=True)

# Bib 재설정 (1부터 시작)
df_merged['Bib'] = range(1, len(df_merged) + 1)

# Bib을 맨 앞으로 이동
cols = df_merged.columns.tolist()
cols.remove('Bib')
df_merged = df_merged[['Bib'] + cols]

# 결측치 확인 (추가적인 안전 확인)
print("결측치 존재 여부:\n", df_merged.isnull().sum().sum())  # 0이면 OK

# Dataset별 샘플 수 확인
print("Dataset 분포:\n", df_merged['Dataset'].value_counts())

# 저장
df_merged.to_csv('./data/combined_Marathon_Data.csv', index=False)
print(f"✔️ 병합된 데이터 저장 완료! 총 샘플 수: {len(df_merged)}")

결측치 존재 여부:
 17222
Dataset 분포:
 Dataset
B    79057
M     8611
Name: count, dtype: int64
✔️ 병합된 데이터 저장 완료! 총 샘플 수: 87668


In [ ]:
import pandas as pd

# 파일 경로
boston_path = './data/boston_processed.csv'
moscow_path = './data/Moscow_Marathon_Processed.csv'

# 데이터 불러오기
df_boston = pd.read_csv(boston_path)
df_moscow = pd.read_csv(moscow_path)

# 데이터 병합
df_merged = pd.concat([df_boston, df_moscow], ignore_index=True)

# 결측치 제거
df_merged.dropna(inplace=True)

# 정수형 변환 (Country, Dataset 제외)
exclude_cols = ['Country', 'Dataset']
int_cols = df_merged.columns.difference(exclude_cols)
df_merged[int_cols] = df_merged[int_cols].astype(int)

# Bib 재설정 및 정렬
df_merged['Bib'] = range(1, len(df_merged) + 1)
cols = df_merged.columns.tolist()
cols.remove('Bib')
df_merged = df_merged[['Bib'] + cols]

# 저장
df_merged.to_csv('./data/combined_Marathon_Data.csv', index=False)

print("병합된 데이터 샘플 수:", len(df_merged))
print("결측치 확인:", df_merged.isnull().sum().sum())

병합된 데이터 샘플 수: 79057
결측치 확인: 0


In [2]:
import pandas as pd

# 각 데이터셋 불러오기
combined_df = pd.read_csv("./data/Combined_Marathon_Data.csv")      # 보스턴 등 기존 데이터
chicago_df = pd.read_csv("./data/chicago_data_processed.csv")       # 시카고 데이터

# 두 데이터셋을 하나로 병합 (인덱스 재정렬 포함)
merged_df = pd.concat([combined_df, chicago_df], ignore_index=True)

# (선택) 저장하고 싶다면
merged_df.to_csv("./data/merged_marathon_data.csv", index=False)


In [ ]:
len(df_moscow)

8611

In [ ]:
import numpy as np

# 수치형 컬럼만 선택
numeric_df = df_merged.select_dtypes(include=[np.number])

# NaN 또는 inf 값이 있는 행의 마스크
non_finite_mask = ~np.isfinite(numeric_df)

# 마스크로 해당 행 추출
rows_with_nan_or_inf = df_merged[non_finite_mask.any(axis=1)]

# 결과 출력
print(f"NaN 또는 inf 값을 포함한 행 수: {len(rows_with_nan_or_inf)}")
display(rows_with_nan_or_inf)

NaN 또는 inf 값을 포함한 행 수: 0


,Bib,Age_group,M/F,Country,5p,10p,15p,20p,25p,30p,...,Sub,5K,10K,15K,20K,25K,30K,35K,40K,Dataset


In [ ]:
count_M = (df_merged['Dataset'] == 'M').sum()
count_B = (df_merged['Dataset'] == 'B').sum()

print(f"Dataset = 'M'인 행 수: {count_M}")
print(f"Dataset = 'B'인 행 수: {count_B}")

Dataset = 'M'인 행 수: 0
Dataset = 'B'인 행 수: 79057


In [ ]:
import numpy as np

# NaN 또는 inf 값이 있는 행을 찾기 위한 마스크 생성
non_finite_mask = ~np.isfinite(df_merged)

# 마스크를 사용하여 해당 행만 추출
rows_with_nan_or_inf = df_merged[non_finite_mask.any(axis=1)]

# 결과 출력
print(f"NaN 또는 inf가 포함된 행 수: {len(rows_with_nan_or_inf)}")
display(rows_with_nan_or_inf)

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''